In [1]:
import torch.nn as nn
import torch
import tqdm.notebook as tqdm
from IPython import display
import matplotlib.pyplot as plt
import pickle as pkl
from sklearn.covariance import EllipticEnvelope
from utils.dataset import load_compiled_data, ObjectDetectorDataset
from utils.head import input_reg, auxiliary_head, mlp_dual, LSTM_DE
from torch.utils.data import Dataset, DataLoader
import random

device = torch.device('cuda:0')
reg_camera_1 = input_reg().to(device)
reg_camera_2 = input_reg().to(device)
aux_head = auxiliary_head().to(device)

reg_camera_1.load_state_dict(torch.load('models/head/reg_camera_1.pt'))
reg_camera_2.load_state_dict(torch.load('models/head/reg_camera_2.pt'))
aux_head.load_state_dict(torch.load('models/head/aux_head.pt'))

reg_camera_1.eval()
reg_camera_2.eval()
aux_head.eval()

aircraft-detection custom utils


auxiliary_head(
  (fc_1): Linear(in_features=256, out_features=128, bias=True)
  (fc_2): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)

In [2]:
norm_offset = [[20.7227, 271.4375,   9.0000,   5.5000], [5.3496, 248.5000,  10.1094,   4.5000]]
norm_divider = [[1894.2773,  798.0625,  174.2500,   67.5000], [1908.6504,  396.5000,   99.8906,   42.5000]]

train_data  = load_compiled_data(ts=[1,2,5],ws=[1],rs=[1,2], convert = True,  offset = norm_offset, divider = norm_divider, device = device)
test_data = load_compiled_data(ts=[1,2,5],ws=[1],rs=[3], convert = True,  offset = norm_offset, divider = norm_divider, device = device)

Data Statistic:
            Camera 1:
            Mean: [0.19101892411708832, 0.04624661058187485, 0.23863928020000458, 0.161183699965477]
            Std : [0.24890540540218353, 0.2908424735069275, 0.25386908650398254, 0.2399858832359314]
            
            Camera 2:
            Mean: [0.22113540768623352, 0.12611186504364014, 0.26530587673187256, 0.1844675987958908]
            Std : [0.25682532787323, 0.42740118503570557, 0.2785913050174713, 0.2537324130535126]
            
Data Statistic:
            Camera 1:
            Mean: [0.17446725070476532, -0.012509922496974468, 0.19903448224067688, 0.12914852797985077]
            Std : [0.2501801550388336, 0.3267986476421356, 0.2563011944293976, 0.2437313199043274]
            
            Camera 2:
            Mean: [0.2224167436361313, 0.10093619674444199, 0.26754945516586304, 0.19287000596523285]
            Std : [0.25625601410865784, 0.41074901819229126, 0.2785138487815857, 0.2622748017311096]
            


# Dropout Training

In [3]:
sequential_de = LSTM_DE().cuda()
sequential_de.train()
tlosses_deep = []
tlosses_shallow = []

In [4]:
learning_rate = 1e-5
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(sequential_de.parameters(), lr=learning_rate)

In [5]:
train_x = torch.stack((reg_camera_1(train_data['dual']['cam_1']['x']), reg_camera_2(train_data['dual']['cam_2']['x'])), dim = 1).detach()
train_y = torch.stack((train_data['dual']['cam_1']['y'][:, 0], train_data['dual']['cam_2']['y'][:, 0]), dim = 1)

test_2cam_x = torch.stack((reg_camera_1(test_data['dual']['cam_1']['x']), reg_camera_2(test_data['dual']['cam_2']['x'])), dim = 1)
test_2cam_y = torch.stack((test_data['dual']['cam_1']['y'][:, 0], test_data['dual']['cam_2']['y'][:, 0]), dim = 1)
test_1cam_x = torch.cat((reg_camera_1(test_data['single']['cam_1']['x']), reg_camera_2(test_data['single']['cam_2']['x']))).unsqueeze(1)
test_1cam_y = torch.cat((test_data['single']['cam_1']['y'][:, 0], test_data['single']['cam_2']['y'][:, 0])).unsqueeze(1)

In [6]:
od_dataset = ObjectDetectorDataset(train_x, train_y)
od_dataloader = DataLoader(od_dataset, batch_size = 32, shuffle=True)

torch.Size([26676, 2, 256]) torch.Size([26676, 2])


In [8]:
# Deep 2 Camera Training
sequential_de.train()
epochs = tqdm.tqdm(iterable=range(16000), leave=True)
for epoch in epochs:
    for train_minibatch_x, train_minibatch_y in od_dataloader:
        optimizer.zero_grad()
        
        drop_choice = random.randint(0,3)
        
        if drop_choice == 0:
            train_minibatch_x = train_minibatch_x[:, 0:1, :]
        elif drop_choice == 1:
            train_minibatch_x = train_minibatch_x[:, 1:, :]
        else:
            pass
        
        ypred = sequential_de(train_minibatch_x)[:, -1].squeeze()
        loss = criterion(ypred, train_minibatch_y[:, 0]) * 10
        loss.backward()
        optimizer.step()
        
    if epoch % 1 == 0:
        sequential_de.eval()
        ypred_shallow = sequential_de(test_1cam_x).squeeze()
        tloss_shallow = criterion(ypred_shallow, test_1cam_y[:, 0]) * 10
        ypred_deep = sequential_de(test_2cam_x).squeeze()[:, 1]
        tloss_deep = criterion(ypred_deep, test_2cam_y[:, 1]) * 10

        test_deep_mape = ((abs(ypred_deep-test_2cam_y[:, 1])/ypred_deep).mean() * 100).item()
        test_shallow_mape = ((abs(ypred_shallow-test_1cam_y[:, 0])/ypred_shallow).mean() * 100).item()
        
        test_deep_e_max = (abs(ypred_deep-test_2cam_y[:, 1])).max() * 10
        test_shallow_e_max = (abs(ypred_shallow-test_1cam_y[:, 0])).max() * 10
        
        display.clear_output(wait=True)
        display.display(epochs.container)
        print("==========================\nEpoch {}".format(epoch))
        print(f'Train Loss           : {loss.item()}\n')

        print(f'Test Shallow Loss    : {tloss_shallow.item()}')
        print(f'Test Deep Loss       : {tloss_deep.item()}\n')

        print(f'Test Shallow MAPE    : {test_shallow_mape:.2f}%')
        print(f'Test Deep MAPE       : {test_deep_mape:.2f}%\n')
        
        print(f'Test Shallow MAPE    : {test_shallow_e_max:.2f} nm')
        print(f'Test Deep MAPE       : {test_deep_e_max:.2f} nm')
    

        print("==========================\n")

        tlosses_deep.append(test_deep_mape)
        tlosses_shallow.append(test_shallow_mape)
        sequential_de.train()

  0%|          | 3/16000 [00:09<10:12:45,  2.30s/it]

Epoch 3
Train Loss           : 8.535169399692677e-06

Test Shallow Loss    : 3.159100378979929e-05
Test Deep Loss       : 1.9246301235398278e-05

Test Shallow MAPE    : 0.16%
Test Deep MAPE       : 0.15%

Test Shallow MAPE    : 0.16 nm
Test Deep MAPE       : 0.09 nm



KeyboardInterrupt: 

In [9]:
torch.save(sequential_de.state_dict(), 'models/head/sequential_de_random_training.pt')

In [7]:
sequential_de.load_state_dict(torch.load('models/head/sequential_de_random_training.pt'))

<All keys matched successfully>

# Old OD

In [16]:
norm_offset = [[20.7227, 271.4375,   9.0000,   5.5000], [5.3496, 248.5000,  10.1094,   4.5000]]
norm_divider = [[1894.2773,  798.0625,  174.2500,   67.5000], [1908.6504,  396.5000,   99.8906,   42.5000]]
train_data = load_compiled_data(ts=[1,2,5],ws=[1],rs=[1,2], offset = norm_offset, divider = norm_divider, device = device)
test_test = load_compiled_data(ts=[1,2,5],ws=[1],rs=[3], offset = norm_offset, divider = norm_divider, device = device)

Data Statistic:
          Camera 1:
          Mean: [0.19101892411708832, 0.04624661058187485, 0.23863928020000458, 0.161183699965477]
          Std : [0.24890540540218353, 0.2908424735069275, 0.25386908650398254, 0.2399858832359314]
          
          Camera 2:
          Mean: [0.22113540768623352, 0.12611186504364014, 0.26530587673187256, 0.1844675987958908]
          Std : [0.25682532787323, 0.42740118503570557, 0.2785913050174713, 0.2537324130535126]
          
Data Statistic:
          Camera 1:
          Mean: [0.17446725070476532, -0.012509922496974468, 0.19903448224067688, 0.12914852797985077]
          Std : [0.2501801550388336, 0.3267986476421356, 0.2563011944293976, 0.2437313199043274]
          
          Camera 2:
          Mean: [0.2224167436361313, 0.10093619674444199, 0.26754945516586304, 0.19287000596523285]
          Std : [0.25625601410865784, 0.41074901819229126, 0.2785138487815857, 0.2622748017311096]
          


In [36]:
train_x = torch.cat((train_data['dual']['cam_1']['x'], train_data['dual']['cam_2']['x']), dim = 1).detach()
train_y = train_data['dual']['cam_1']['y'][:, 0]

test_2cam_x = torch.cat((test_data['dual']['cam_1']['x'], test_data['dual']['cam_2']['x']), dim = 1)
test_2cam_y = test_data['dual']['cam_1']['y'][:, 0]

In [31]:
model = mlp_dual().cuda()

In [32]:
learning_rate = 1e-4
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [41]:
# old od training
sequential_de.train()
losses = []
tlosses = []
epochs = tqdm.tqdm(iterable=range(700000), leave=True)
for epoch in epochs:
    optimizer.zero_grad()
    # camera_1
    ypred = model(train_x).squeeze()
    loss = criterion(ypred, train_y) * 100
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        sequential_de.eval()
        ypred = model(test_2cam_x).squeeze()
        tloss = criterion(ypred, test_2cam_y) * 100
        
        display.clear_output(wait=True)
        #display.display(epochs.container)
        print("==========================\nEpoch {}".format(epoch))
        print(f'Train Deep Loss      : {loss.item()}')
        print(f'Test Deep Loss       : {tloss.item()}')
        print(f'Test Deep MAPE       : {((abs(ypred-test_2cam_y)/ypred).mean() * 100).item():.2f}%')
        
        print("==========================\n")
        #losses.append([loss.item()])
        #tlosses.append([tloss.item()])
        sequential_de.train()

Epoch 500
Train Deep Loss      : 38.06881332397461
Test Deep Loss       : 36.15445327758789
Test Deep MAPE       : inf%



KeyboardInterrupt: 